水气压（或水汽压）是指空气中水蒸气的分压力。

In [ ]:
import ee
import geemap
import os
import math
import geopandas as gpd

# 初始化 Earth Engine
ee.Initialize()

In [ ]:
# 读取本地Shapefile 得到处理区域
shapefile_path = r"F:\ArcgisData\shp_polygon\SB500.shp"
gdf = gpd.read_file(shapefile_path)


bounds = gdf['geometry'][0].bounds
bbox = {
    'MinX': bounds[0],
    'MinY': bounds[1],
    'MaxX': bounds[2],
    'MaxY': bounds[3]
}

# 获取第一个几何对象
geometry = gdf.geometry.iloc[0]
# 2. 将 Earth Engine 几何对象转换为 Feature 对象
ee_geometry = ee.Geometry(geometry.__geo_interface__)
# 3. 将 Earth Engine 几何对象转换为 Feature 对象
feature = ee.Feature(ee_geometry)

# 4. 将 Feature 对象放入 FeatureCollection 中
feature_collection = ee.FeatureCollection([feature])

# 获取 FeatureCollection 的边界几何对象
bounded_geometry = feature_collection.geometry().bounds()
# 矢量边框
vector_layer = ee.FeatureCollection([ee.Feature(ee_geometry)])

# 定义要下载的图像的参数
area_of_interest =ee.Geometry.Rectangle([bbox['MinX'], bbox['MinY'], bbox['MaxX'], bbox['MaxY']])  # 请替换经度和纬度

In [ ]:


# 定义区域
region = area_of_interest

# 获取 ERA5 气温数据集
# temperature = ee.ImageCollection('ECMWF/ERA5/DAILY')
temperature = ee.ImageCollection('ECMWF/ERA5_LAND/DAILY_AGGR')
temp2m = temperature.select('dewpoint_temperature_2m')

# 获取全年每日的最高和最低气温
max_temp = temp2m.filter(ee.Filter.calendarRange(1, 12, 'month')).max()
min_temp = temp2m.filter(ee.Filter.calendarRange(1, 12, 'month')).min()

# 计算年温差
annual_temp_range = max_temp.subtract(min_temp)

# 获取 ERA5 湿度数据集
humidity = temperature.select('dewpoint_temperature_2m')

# 计算水气压
def compute_vapor_pressure(image):
    temp = image.select('dewpoint_temperature_2m')
    dewpoint = image.select('dewpoint_temperature_2m')
    e_s = temp.expression('6.112 * exp((17.67 * temp) / (temp + 243.5))', {'temp': temp.subtract(273.15)})
    e = dewpoint.expression('RH / 100 * e_s', {'RH': dewpoint, 'e_s': e_s})
    return e.rename('vapor_pressure')

vapor_pressure = temperature.map(compute_vapor_pressure).mean()

# 可视化参数
visualization_temp_range = {'min': 0, 'max': 40, 'palette': ['blue', 'green', 'red']}
visualization_vapor_pressure = {'min': 0, 'max': 40, 'palette': ['blue', 'green', 'red']}

# 创建地图对象并添加图层
m = geemap.Map()
m.addLayer(annual_temp_range, visualization_temp_range, 'Annual Temperature Range')
m.addLayer(vapor_pressure, visualization_vapor_pressure, 'Vapor Pressure')
m.addLayerControl()
# 显示地图
m
